In [69]:
import folium
import pandas as pd
df = pd.read_csv('restaurant_list.csv', encoding='utf-8')
df.columns= ['id',
             'name',
             'cate',
             'dong',
             'lat',
             'lon',
             'score',
             'review',
             'contact',
             'address',
             ]
df

,id,name,cate,dong,lat,lon,score,review,contact,address
0,1292414361,카페 드 파리,카페,명동,37.562711,126.983746,4.70,23,010-2216-0020,서울 중구 명동4길 26 2층
1,1798095400,몰또이탈리안에소프레소바,카페,명동,37.564577,126.986213,3.40,144,02-778-7779,서울 중구 명동길 73 3층
2,26934440,르빵명동성당점,카페,명동,37.563240,126.987005,3.60,84,02-3789-7545,서울 중구 명동길 74 B103호
3,1911581917,가배도명동,카페,명동,37.560576,126.985138,4.20,333,0507-1304-4542,서울 중구 퇴계로18길 10 1층
4,474005666,스타벅스레스케이프호텔R점,카페,명동,37.560091,126.979491,4.30,221,1522-3232,서울 중구 퇴게로 67 7층
...,...,...,...,...,...,...,...,...,...,...
107,1335913739,금산제면소,한식,명동,37.558630,126.983679,1.90,225,-,서울 중구 소공로6길 13
108,254918904,다올 숯불구이,한식,명동,37.475601,126.937821,3.74,417,0507-1462-1246,서울 중구 명동8길 8-11 1 2층
109,549474667,야마야 강남N타워점,일식,강남,37.499799,127.032435,3.30,21,02-6250-0980,서울 강남구 테헤란로 129 강남N타워 1층
110,26816485,강남꿀돼지,한식,강남,37.514442,127.064657,3.00,1,02-508-8552,서울 강남구 봉은사로108길 3-14


In [70]:
map = folium.Map(location=[37.561142, 126.986133], # 명동역
                 tiles="CartoDB positron",
                 zoom_start=14)

In [71]:
from folium.plugins import MarkerCluster
from folium import Marker

# 밀집되어 있을 시에 보기 불편함을 해소해주기 위해 주변지역을 묶어서 대략적인 개수 분포 보기
mc = MarkerCluster()

# 평점별 다른 색으로 마커 표시
def color_select(row):
    score = float(row.score)
    if score >= 4.7:
        return 'blue'
    elif score >= 4.3:
        return 'green'
    else:
        return 'beige'
# 카테고리별 다른 아이콘으로 마커 표시
def cate_select(row):
    if row.cate.strip() in ['한식', '일식', '중식']:
        return 'bowl-food'
    elif row.cate == '양식':
        return 'utensils'
    else:
        return 'mug-saucer'

for i, r in df.iterrows():
    place_id = str(r['id'])
    name = str(r['name'])
    popup_html = f'<a href="https://place.map.kakao.com/{place_id}">{name}[상세 정보 보기]</a><br>'
    popup_html += f'''<div style="text-align:center;"><form><input type="button" value="밥 친구 구하러 가기" onclick="handleCreateBtn('{name}')"></form></div>'''
    mc.add_child(
        Marker(location = [r['lat'], r['lon']],
               tooltip = r['name'],
               popup=folium.Popup(popup_html,
                                  max_width=1000),
               icon=folium.Icon(color = color_select(r),icon=cate_select(r), prefix='fa')
              )
    )

map.add_child(mc)



In [72]:
from folium import plugins

# 유저 실시간 위치 파악
plugins.LocateControl().add_to(map)
map.save('recommend.html')
